In [9]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import os
import pandas as pd
import numpy as np

import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from metric_learn import NCA
from sklearn.model_selection import cross_val_score, train_test_split

from sklearn.svm import SVC

from gensim.models import FastText
from gensim.test.utils import common_texts
from tpot import TPOTClassifier

import sys
sys.path.append(f'../../')

from src.data.sentence_vectorizer import SentenceVectorizer

In [3]:
#ft_model = FastText.load_fasttext_format('BioWordVec_PubMed_MIMICIII_d200.bin')
#ft_model

In [ ]:
model = FastText.

In [8]:
labels = pd.read_csv('../../data/interim/meddra_codes_terms_synonims.csv')
labels = labels['CODE']
meddra_labels = {v:k for k, v in enumerate(labels.unique())}

results = {
    'vectorizer': [],
    'train_model': [],
    'smm4h21': [],
    'smm4h17': [],
    'psytar': [],
    'cadec': [],
}

SIZE = 100

for vectorizer_name in sv.get_availables_vectorizers():
    print(f"vectorizer: {vectorizer_name}")
    results['vectorizer'].append(vectorizer_name)
    path = '../../data/interim/'
    for name_train in os.listdir(path):

        if name_train not in ['smm4h17', 'smm4h21', 'psytar', 'cadec']:
            continue
        print(f"work with {name_train}")
        results['train_model'].append(name_train)

        folder = os.path.join(path, name_train)
        corpus_train = folder + '/train.csv'
        corpus_test = folder + '/test.csv'
        train, test = pd.read_csv(corpus_train), pd.read_csv(corpus_test) 
        sv = SentenceVectorizer()
        train, test = sv.vectorizer(train, test, vectorizer_name=vectorizer_name) 

    #     print('orig', cadec_train.shape, cadec_test.shape)
    #     print('vectorized', train.shape, test.shape)

        X_train, y_train = train['term_vec'], train['code']
        X_test, y_test = test['term_vec'], test['code']

        X_train = pd.DataFrame([pd.Series(x) for x in X_train])
        X_test = pd.DataFrame([pd.Series(x) for x in X_test])

        y_train = y_train.apply(lambda x: int(meddra_labels[x]))
        y_test = y_test.apply(lambda x: int(meddra_labels[x]))

    #     print('model input:', X_train.shape, y_train.shape, X_test.shape, y_test.shape)

        #clf = make_pipeline(NCA(), SVC(gamma='scale'))
        clf = SVC(gamma='scale')
        clf.fit(X_train, y_train)

        for name_test in os.listdir(path):
            if name_test not in ['smm4h17', 'smm4h21', 'psytar', 'cadec']:
                continue
            folder = os.path.join(path, name_test)
            corpus_train = folder + '/train.csv'
            corpus_test = folder + '/test.csv'
            train, test = pd.read_csv(corpus_train), pd.read_csv(corpus_test) 
            sv = SentenceVectorizer()
            train, test = sv.vectorize_sent_ft(train, test, 
                                               feat_col='term', 
                                               text_columns = ['term', 'text', 'SNMS'],
                                               size=SIZE) 
            X_test, y_test = test['term_vec'], test['code']
            X_test = pd.DataFrame([pd.Series(x) for x in X_test])
            y_test = y_test.apply(lambda x: int(meddra_labels[x]))

            score = clf.score(X_test, y_test)
            print(f'test with {name_test} score:', score)
            results[name_test].append(score)
        print()
    
results = pd.DataFrame(results)
results

SyntaxError: invalid syntax (<ipython-input-8-ef065c8c5732>, line 7)

In [4]:
results # size 50

,train_model,smm4h21,smm4h17,psytar,cadec
0,smm4h21,0.355685,0.144458,0.114035,0.137344
1,smm4h17,0.186589,0.777911,0.290727,0.257662
2,psytar,0.134111,0.286114,0.598997,0.203178
3,cadec,0.137026,0.146058,0.186717,0.598184


In [17]:
generations = 2
population_size = 50
max_eval_time_mins = 2
n_jobs = 10
max_iter = 10

tpot = TPOTClassifier(generations=generations, 
                      population_size=population_size,
                      verbosity=2, 
                      random_state=42, 
                      max_eval_time_mins=max_eval_time_mins, 
                      n_jobs=n_jobs)

tpot.fit(X_train, y_train)
score = tpot.score(X_test, y_test)
score

Optimization Progress:   0%|          | 0/150 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.5222418453598638

Generation 2 - Current best internal CV score: 0.5222418453598638

Best pipeline: RandomForestClassifier(input_matrix, bootstrap=False, criterion=gini, max_features=0.05, min_samples_leaf=1, min_samples_split=7, n_estimators=100)


0.5225563909774437